In [40]:
import torch
import torch.nn as nn

1. corr2d 함수 구현

In [41]:
def corr2d(X, K):
    # Compute 2D cross-correlation
    # X: input tensor, K: kernel tensor
    # Y: output tensor
    [R,C] = X.shape # R ; input tensor row , C : input tensor column
    [r,c] = K.shape # r : kernel tensor row , c : kernel tensor column
    stride = 1 # default stride를 1로 세팅
    output_sizeR = int((R-r)/stride + 1) # Y의 row size 계산
    output_sizeC = int((C-c)/stride + 1) # Y의 column size 계산
    Y = torch.zeros(output_sizeR, output_sizeC) # 0으로 구성된 Y 생성
    for i in range(output_sizeR):
      for j in range(output_sizeC):
        for k in range(r):
          for l in range(c):
            Y[i,j] = Y[i,j] + X[i+k,j+l]*K[k,l] #cross correlation 계산
    return Y

In [42]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]]) # Input tensor
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]]) # Kernel tensor

corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

2. pad_2d 함수 구현

In [43]:
def pad_2d(X, P):
    # Compute 2D padding
    # X: input tensor, P: padding size
    [R,C] = X.shape # R ; input tensor row , C : input tensor column
    r_pad_size = P[0]
    r_pad = torch.zeros(r_pad_size, C)
    Y_yet = torch.cat((r_pad, X, r_pad))
    [nR, nC] = Y_yet.shape
    c_pad_size = P[1]
    c_pad = torch.zeros(nR, c_pad_size)
    Y = torch.cat((c_pad, Y_yet, c_pad),1)
    return Y

In [45]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]]) # Input tensor
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]]) # Kernel tensor

X_pad = pad_2d(X, (1, 1)) # padding size (1, 1)
X_pad.shape
print(X_pad)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 1., 2., 0.],
        [0., 3., 4., 5., 0.],
        [0., 6., 7., 8., 0.],
        [0., 0., 0., 0., 0.]])


3. corr2d_str 함수 구현

In [46]:
import math

def corr2d_str(X, K, S):
    # Compute 2D cross-correlation with stride
    # X: input tensor, K: kernel tensor
    # S: stride
    # Y: output tensor
    [R,C] = X.shape # R ; input tensor row , C : input tensor column
    [r,c] = K.shape # r : kernel tensor row , c : kernel tensor column
    S_r = S[0]
    S_c = S[1]
    output_sizeR = int((R-r)/S_r+ 1) # Y의 row size 계산
    output_sizeC = int((C-c)/S_c + 1) # Y의 column size 계산
    Y = torch.zeros(output_sizeR, output_sizeC) # 0으로 구성된 Y 생성
    for i in range(output_sizeR):
      for j in range(output_sizeC):
        for k in range(r):
          for l in range(c):
            Y[i,j] = Y[i,j] + X[i+k,j+l]*K[k,l] #cross correlation 계산
    
    return Y

In [47]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]]) # Input tensor
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]]) # Kernel tensor

X_pad = pad_2d(X, (1, 1)) # padding size (1, 1)
X_pad.shape

corr2d_str(X_pad, K, (3, 2)) # stride (3, 2)

tensor([[ 0.,  3.],
        [ 9., 19.]])

4. pool2d 함수 구현

In [48]:
def pool2d(X, pool_size, mode='max'):
    # Compute 2D pooling
    # X: input tensor, pool_size: pooling size, mode: 'max' or 'avg'
    # Y: output tensor
    [R,C] = X.shape # R ; input tensor row , C : input tensor column
    r = pool_size[0]
    c = pool_size[1]
    p = r*c
    output_sizeR = int((R-r) + 1) # Y의 row size 계산
    output_sizeC = int((C-c) + 1) # Y의 column size 계산
    Y = torch.zeros(output_sizeR, output_sizeC)
    if mode == 'max':
      dummy = torch.zeros(r,c)
      for i in range(output_sizeR):
        for j in range(output_sizeC):
          for k in range(r):
            for l in range(c):
              dummy[k,l] = X[i+k,j+l]
          Y[i,j] = torch.max(dummy)

    if mode == 'avg':
      for i in range(output_sizeR):
        for j in range(output_sizeC):
          for k in range(r):
            for l in range(c):
              Y[i,j] = Y[i,j] + X[i+k,j+l]
      Y = Y/p

    return Y

In [49]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])

pool2d(X, (2, 2), 'max') # 2 X 2 max pooling

tensor([[4., 5.],
        [7., 8.]])

In [50]:
pool2d(X, (2, 2), 'avg') # 2 X 2 average pooling

tensor([[2., 3.],
        [5., 6.]])